In [ ]:
#using DifferentialEquations
using Plots, Colors, LaTeXStrings
using Plots.Measures
font_title = Plots.font("Arial", 24)
font_axis = Plots.font("Arial", 12)
font_legend = Plots.font("Arial", 8)
pyplot(titlefont=font_title, guidefont = font_axis, legendfont = font_legend)

#Define different symbols used in the plotting
delta = latexstring("\$\\delta\$")
micro = latexstring("\$\\mu\$")
vt = latexstring("\$V_t\$")
nt = latexstring("\$N_t\$")
bt = latexstring("\$B_t\$")
wt = latexstring("\$W_t\$")
i_ach = latexstring("\$I_{ACh}\$")

v_color = :deepskyblue
n_color = :magenta
c_color = :green
a_color = :purple
b_color = :red
e_color = :blue
w_color = :gray

figure_path = "C:/users/mtarc/JuliaScripts/RetinalChaos/Notebooks/Figures"

In [ ]:
using StatsBase, Statistics, Distributions, StatsPlots

# RetinalChaos.jl: An introduction
### In order to understand the code published here, these notebooks will act as simple tutorials on both how the model was designed. 

## Table of contents:

### _**[1] Introduction, Motivation, and Components**_
#### [1.1] Voltage and Potassium gating components of the model
#### [1.2] Calcium and the Biochemical Reactions of the sAHP
#### [1.3] Acetylcholine Diffusion and Dynamics
#### **[1.4] Gaussian White noise and dynamics**

### [2] Analyzing Data Output of the model
#### [2.1] Dynamical Analysis
#### [2.2] Fitting Experimental Data (Patch)
#### [2.3] Fitting Experimental Data (Multi-electrode array)
#### [2.4] Fitting Experimental Data (Calcium Imageing) 

### [3] Experiments
#### [3.1] Dual Eye Correlations
#### [3.2] Recapitulations of older papers

# Just to recap the model in full is 
\begin{align}
   C_m\frac{dV}{dt} &= I_{leak}(V_t) + I_{Ca}(V_t) + I_K(V_t, N_t) + I_{TREK}(V_t, B_t) + I_{ACh}(V_t, A_t) + I_{app} + \sigma W_t\\
   \tau_N\frac{dN}{dt} &= \Lambda(V_t)(N_\infty(V_t) - N_t)\\
   \tau_C\frac{dC}{dt} &= C_0 + \delta I_{Ca}(V_t) -\lambda  C_t\\
   \tau_A\frac{dA}{dt} &= \alpha * C_t^4 * \left(1- A_t\right) - A_t\\
   \tau_B\frac{dB}{dt} &= \beta * A_t^4 * (1 - B_t) - B_t\\
   \tau_E\frac{dE}{dt} &=  D\nabla^2 E_t + \rho \Phi(V_t) - E_t\\
   \tau_W\frac{dW}{dt} &= -W_t
\end{align}


### Adding In noise to the model

In [ ]:
import RetinalChaos: read_JSON, extract_dict
import RetinalChaos: T_sde
import RetinalChaos: SDEProblem, solve, SOSRI

In [ ]:
import RetinalChaos: ODEProblem, T_ode, codim_map

In [ ]:
p_dict = read_JSON("params.json");
p_dict[:σ] = 0.1
p_dict[:τw] = 1000.0
p = p_dict |> extract_dict;
u0 = read_JSON("conds.json") |> extract_dict;
tspan = (0.0, 300e3);
dt = 1.0
SDEprob = SDEProblem(T_sde, u0, tspan, p)
println("Time it took to simulate 200ms:")
@time SDEsol = solve(SDEprob, SOSRI(), abstol = 2e-2, reltol = 2e-2, maxiters = 1e7, saveat = dt); 

In [ ]:
xlims = (100e3, 160e3) 
xticks = (collect(xlims[1]:5000:xlims[2]), collect(0:5.0:(xlims[1]-xlims[2]/1000)))
fig3_A = plot(SDEsol,
    xticks = xticks, 
    xlims = xlims,
    vars = [:v, :b, :W], 
    xlabel = ["" "" "Time (s)"],
    ylabel = ["$vt (mV)" "$bt" "$wt (pA)"],
    labels = ["" "" ""],
    c = [v_color b_color w_color], 
    lw = 2.0,
    layout = grid(3, 1))
title!(fig3_A[1], "A", title_location = :left);

In [ ]:
xlims = (-15.0, 15.0); 
Wt = map(t->SDEsol(t)[7], SDEsol.t);
fig3_B = histogram(Wt, c = :gray, 
    xlabel = "Noise (pA)", 
    ylabel = "Probability of occurance", 
    label = "", 
    normalize = :pdf, 
    xlims = xlims
    #orientation = :h
)
noise_fit = fit(Normal, Wt)
mu = round(noise_fit.μ, digits = 2)
sig = round(noise_fit.σ, digits = 2)
plot!(fig3_B, noise_fit, c = :black, 
    #orientation = :h,
    label = "Fit (μ = $mu pA σ = $sig pA)",
    linestyle = :dash, lw = 4.0, size = (500,500))
title!(fig3_B, "B", title_location = :left);

In [ ]:
#Make a problem, specifying all parameters
p_dict = read_JSON("params.json") 
p_dict[:I_app] = 10.0
p_dict[:g_ACh] = 0.0 #Remove g_ACh influence
p_dict[:g_TREK] = 0.0 #Remove g_TREK influence
p = p_dict |> extract_dict;
u0 = read_JSON("conds.json") |> extract_dict;
tspan = (0.0, 30000);
prob = ODEProblem(T_ode, u0, tspan, p)
codim1 = (:I_app)

@time c1_map = codim_map(prob, codim1, c1_lims = xlims, eq_res = 10);
fit_rng = collect(xlims[1]:0.1:xlims[2]);
vline(fit_rng, line_z = pdf.(noise_fit, fit_rng).*100, c = :YlOrBr_8, lw = 10.0, alpha = 0.5, label = "Noise Probability", ylims = (-80.0, -20.0), cbar = false)
plot!(c1_map, markersize = 10, xlabel = "\$I_{App}\$ (pA)", ylabel = "\$V_t\$ (mV)", grid = false)
fig3_C = title!("C", title_location = :left);

In [ ]:
fig3 = plot(fig3_A, fig3_B, fig3_C, grid = false, layout = grid(3,1), size = (1000,1000))

In [ ]:
savefig(fig3, "$(figure_path)/Figure3_Gaussian_White_Noise.png")

In [ ]:
import RetinalChaos: ensemble_func, EnsembleProblem, EnsembleThreads

In [ ]:
n_sims = 5
par_sym = :τw
par = findall(isequal(par_sym), Symbol.(T_sde.ps))[1]
#or 
#This is just a repeated measures trial
par = :none
p_range = LinRange(0.01, 5.0, n_sims)
println(par)

In [ ]:
prob_func(prob, i, repτ = ensemble_func(prob, i, repeat; pars = par, rng = p_range)
ensemble_prob = EnsembleProblem(SDEprob, prob_func = prob_func)

In [ ]:
@time sim = solve(ensemble_prob, SOSRI(), abstol = 0.2, reltol = 2e-2, maxiters = 1e7, saveat = 100.0, trajectories = n_sims, EnsembleThreads());

In [ ]:
vars = [:v :b :W]
xlims = (0.0, 60e3) 
xticks = (collect(xlims[1]:5000:xlims[2]), collect(0:5.0:(xlims[1]-xlims[2]/1000)))
p = plot(layout = grid(length(vars),1), xlims = (0, 60e3), size = (800, 800))
for (idx, traj) in enumerate(sim)
    val = p_range[idx]
    plot!(p, traj, vars = vars, lw = 1.0, 
        c = :delta, line_z = val, clim = (p_range[1], p_range[end]), legend = :none, 
        xticks = xticks, 
        xlims = xlims
    )
    #plot(p, )
end
p